In [7]:
import aknotebooks.classification.convenience_functions.mkl_base as mkl_base
from aknotebooks.classification.convenience_functions.mkl_base import hardDrivesLoc, dataOnlyDrive, folderList, symbols, \
    dataList, symbolFeaturesLocation, selection,checkDir
from aknotebooks.classification.convenience_functions.fit_mkl import featureCreation

In [8]:
import MKLpy
from MKLpy.metrics import pairwise
from MKLpy.algorithms import AverageMKL, EasyMKL, KOMD#KOMD is not a MKL algorithm but a simple kernel machine like the SVM
#evaluate the solution
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import roc_curve, auc
from MKLpy.metrics import pairwise
from MKLpy.preprocessing import normalization, rescale_01
from MKLpy.preprocessing import kernel_normalization
from MKLpy.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics.pairwise import rbf_kernel as RBF
import os
import pandas as pd
import numpy as np
import pickle as pkl

In [9]:
symbols

['BLT.L',
 'CNA.L',
 'PRU.L',
 'BATS.L',
 'CEY.L',
 'APF.L',
 'AZN.L',
 'CCL.L',
 'CPG.L',
 'BARC.L',
 'AV.L',
 'AAL.L']

In [10]:

symbolIdx = 2  # pick one of the symbols

# do a join to get the location
# symbolLocation = "/".join((finalLocation, symbols[symbolIdx]))
# # get the features now

selectionLoc = os.path.join(hardDrivesLoc, selection)

# ''' location of WorkDrive'''
# dataList = [s for s in os.listdir(selectionLoc) if s.startswith('Dat')]
# path = 'MKL_Experiments'
MKLExpPath = os.path.join(os.path.join(hardDrivesLoc, selection, dataList[1]), 'MKL_Experiments')
MKLSymbolPath = os.path.join(MKLExpPath, symbols[symbolIdx])
MKLSymbolKernelsPath = "/".join((MKLSymbolPath, 'Kernels'))

SymbolCommonPaths = mkl_base.open_pickle_file(MKLSymbolPath, 'LocDictsListCorrect.pkl')
uniqueTrainingKeys = np.unique([i[1] for i in SymbolCommonPaths])

In [46]:
#
 kernelDates = [file.split("_")[0] for file in os.listdir('/media/ak/WorkDrive/Data/MKL_Experiments/PRU.L/Kernels/')

                

In [55]:
kernelDates[0]

'20170131'

In [56]:
keys=list(SymbolCommonPaths.keys())
type(keys)
# for idx, key in enumerate(keys):
#     print(idx)
#     print(key)
#     kernelFileName = "/".join((MKLSymbolKernelsPath, "".join((key[0], "_Kernels.pkl"))))
#     checkDir(kernelFileName)
#     print(kernelFileName)
#     print(SymbolCommonPaths[key])
i=kernelDates[0]
result=[]
for j in keys:
    if i in j:
        result.append(j)

print(result)

[('20170120', '20170131'), ('20170117', '20170131')]


In [59]:
print(SymbolCommonPaths[keys[920]])
kernelFileName = "/".join((MKLSymbolKernelsPath, "".join((i, "_Kernels.pkl"))))
checkDir(kernelFileName)

['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170117/PRU.L_3_states_features_date:_20180406_now:_20181229_.pickle', '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20180406.csv']
The directory exists


In [57]:
SymbolCommonPaths[result[1]]

['/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170117/PRU.L_3_states_features_date:_20170131_now:_20181229_.pickle',
 '/media/ak/DataOnly/FinDataReal/Labels/PRU.L/NON_DIRECTIONAL/20170131.csv']

In [70]:
featuresTupleFile=pkl.load(open(SymbolCommonPaths[result[1]][0], "rb"),encoding='latin1')
labelsIdxDirFileLoc = pd.read_csv(SymbolCommonPaths[result[1]][1])

In [69]:

dfFeatures = pd.concat([featuresTupleFile[0], featuresTupleFile[1], \
                            featuresTupleFile[2], featuresTupleFile[3]], axis=1, sort=False).fillna(0)

In [ ]:
pd.read_csv(labelsIdxDirFileLoc)

In [62]:
testKernes =pkl.load(open(kernelFileName, "rb"))

In [63]:
np.shape(testKernes[0])

(10, 2661, 2661)

In [ ]:
if __name__ == '__main__':
    trainingKey =1 
    keyDate = list(SymbolCommonPaths)[trainingKey]
    
    

    for trainingKey, keyDate in enumerate(list(SymbolCommonPaths)):

        print(trainingKey, keyDate[1])
        Xtr, ytr = featureCreation(idxKey=trainingKey,
                                   locDict=SymbolCommonPaths)  # need to refactor this to read from the mkl_base
        if Xtr.shape[0] == ytr.shape[0]:
            print(list(SymbolCommonPaths.keys())[trainingKey][0])
            kernelFileName = "/".join((MKLSymbolKernelsPath, "".join((keyDate[0], "_Kernels.pkl"))))
            loadPickleKernels = pkl.load(open(kernelFileName, "rb"))
            print('doing the kernels bit')
            KLsimple = loadPickleKernels[0]
            KLrbf = loadPickleKernels[1]
            clf = AverageMKL().fit(KLsimple, ytr)  # a wrapper for averaging kernels
            clfEasy = EasyMKL(lam=0.1).fit(KLsimple, ytr)#combining kernels with the EasyMKL algorithm
            clfRBF = EasyMKL(lam=0.1).fit(KLrbf, ytr)
            print('------')
            print('finished MKL Fitting- Now will test Predictions')
            print('Average Kernel Testing')
            y_pred = clf.predict(KLsimple)  # predictions
            y_score = clf.decision_function(KLsimple)  # rank
            accuracy = accuracy_score(ytr, y_pred)
            fprAverage, tprAverage, thresholdsAverage = roc_curve(ytr.ravel(), y_score.ravel())
            roc_auc = auc(fprAverage, tprAverage)
            print('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))
        else:
            print('Shapes dont match.')
            continue

In [ ]:
trainingKey =1 
keyDate = list(SymbolCommonPaths)[trainingKey]
print(trainingKey, keyDate[1])
Xtr, ytr = featureCreation(idxKey=trainingKey,
                                   locDict=SymbolCommonPaths)  # need to refactor

In [ ]:
Xtr.shape[0] == ytr.shape[0]

In [ ]:
print(list(SymbolCommonPaths.keys())[trainingKey][0])
kernelFileName = "/".join((MKLSymbolKernelsPath, "".join((keyDate[1], "_Kernels.pkl"))))
loadPickleKernels = pkl.load(open(kernelFileName, "rb"))
print('doing the kernels bit')
KLsimple = loadPickleKernels[0]
KLrbf = loadPickleKernels[1]

In [ ]:
clf = AverageMKL().fit(KLsimple, ytr)  # a wrapper for averaging kernels
clfEasy = EasyMKL(lam=0.1).fit(KLsimple, ytr)#combining kernels with the EasyMKL algorithm
            
clfRBF = EasyMKL(lam=0.1).fit(KLrbf, ytr)